In [1]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

In [2]:
from __future__ import print_function

import os
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt
from torch.autograd import Variable
from sklearn.metrics import roc_auc_score, precision_score, recall_score, accuracy_score
from torchsummary import summary

In [3]:
import import_ipynb
from datagen import Data

importing Jupyter notebook from datagen.ipynb


In [4]:

seglength = 120
train_ratio = 0.8
focal_train, focal_test, focal_train_labels, focal_test_labels = Data('D:\EEG\FOCAL', train_ratio, seglength, class_num = 1)
ige_train, ige_test, ige_train_labels, ige_test_labels = Data('D:\EEG\IGE', train_ratio, seglength, class_num = 2)


In [5]:

val_ratio = 0.1
X_train = torch.cat((focal_train, ige_train))
Y_train = torch.cat((focal_train_labels, ige_train_labels))

X_test = torch.cat((focal_train, ige_train))
Y_test = torch.cat((focal_train_labels, ige_train_labels))

X_val = X_train[:int(val_ratio * len(Y_train))].cuda(0)
Y_val = Y_train[:int(val_ratio * len(Y_train))].cuda(0)

X_train = X_train[int(val_ratio * len(Y_train)):].cuda(0)
Y_train = Y_train[int(val_ratio * len(Y_train)):].view(-1, 1).cuda(0)




In [6]:
#del focal_train, focal_test, focal_train_labels, focal_test_labels, ige_train, ige_test, ige_train_labels, ige_test_labels

In [ ]:
np.shape(Y_train)

In [ ]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.T = 9000

        # Layer 1
        self.conv1 = nn.Conv2d(1, 16, (1, 20))
        self.batchnorm1 = nn.BatchNorm2d(16, False)

        # Layer 2
        self.conv2 = nn.Conv2d(16, 32, (5, 1), stride=5)
        self.batchnorm2 = nn.BatchNorm2d(32, False)

        # FC
        self.fc1 = nn.Linear(32*24, 1)

    def forward(self, x):
        # Layer 1
        x = F.relu(self.conv1(x))
        x = self.batchnorm1(x)
        x = F.dropout(x, 0.25)

        # Layer 2
        x = F.relu(self.conv2(x))
        x = self.batchnorm2(x)


        x = x.permute(0, 3, 1, 2)



        x = x.reshape(-1, x.shape[1]* x.shape[2]* x.shape[3])
        x = F.sigmoid(self.fc1(x))
        return x



model = CNN()        

#model.forward(X_train[0:200])
#summary(model.cuda(), (1, 120, 20))
model.forward(X_train[0:2])

tensor([[0.3560],
        [0.3598]], grad_fn=<SigmoidBackward0>)

In [ ]:
batch_size = 4096
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters())

for epoch in range(2):  # loop over the dataset multiple times
    print("\nEpoch ", epoch)
    running_loss = 0.0
    for i in range(int(len(X_train)/batch_size-1)):
        s = i*batch_size
        e = i*batch_size+batch_size
        
        inputs = X_train[s:e]
        labels = Y_train[s:e]



        #inputs, labels = Variable(inputs.cuda(0)), Variable(labels.cuda(0))

        optimizer.zero_grad()
        outputs = model.forward(inputs)

        loss = criterion(outputs, labels)
        loss.backward()

        optimizer.step()

        print(loss.data)


Epoch  0
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(-40.1367)
tensor(-100.)
tensor(-100.)
tensor(-100.)
tensor(-100.)
tensor(-100.)
tensor(-100.)
tensor(-100.)
tensor(-100.)
tensor(-100.)
tensor(-100.)
tensor(-100.)
tensor(-100.)
tensor(-100.)
tensor(-100.)
tensor(-100.)
tensor(-100.)
tensor(-100.)
tensor(-100.)
tensor(-100.)
tensor(-100.)
tensor(-100.)
tensor(-100.)
tensor(-100.)
tensor(-100.)
tensor(-100.)
tensor(-100.)
tensor(-100.)
tensor(-100.)
tensor(-100.)
tensor(-100.)
tensor(-100.)
tensor(-100.)
tensor(-100.)
tensor(-100.)
tensor(-100.)
tensor(-100.)
tensor(-100.)
tensor(-100.)
tensor(-100.)
tensor(-100.)
tensor(-100.)
ten